
# Quiz 3
# Machine Learning 2018-1


Keep saving your notebook to guarantee that you don't loose your work. Whenever the end of the quiz is announced save the current version. 

---



In [1]:
import numpy as np
import pylab as pl

%matplotlib inline

### 1. (2.0)
The following code implements a feed-forward neural network with one hidden layer:

In [2]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(x):
    return max(0, x)

def predict(w, x):
    z = np.zeros((3,))
    z[2] = relu(np.dot(x,w[6:8]) + w[8])
    z[1] = relu(np.dot(x,w[3:5]) + w[5])
    z[0] = sigmoid(np.dot(z[1:3], w[0:2]) + w[2])
    return z[0]

Find a weight vector such that the neural network calculates the negated XOR function:
    
$$f(x,y)=\neg(x\textrm{ xor }y)$$

Use the following variable to put the weight vector:

In [3]:
W1 = np.zeros(9)

$$ f(x,y) = \neg(x\textrm{ xor }y)  = (x \textrm{ and } y) \textrm{ or } (\neg x \textrm{ and } \neg y)$$

In [4]:
#             2           5          8
W1 = [32, 32, -8,   -1, -1, 0.5,   1, 1, -1.5]

W1 = [8, 8, -8,   -4, -4, 2,   2, 2, -2]

X = [[0, 0],
     [0, 1],
     [1, 0],
     [1, 1]]
Y = [1, 0, 0 ,1]

error=0
for i, x in enumerate(X):
    y = Y[i]
    p = predict(W1, x)
    error += np.abs(y-p) 
    print(x, "{} == {:.2}".format(y, p))
    w = W1
    z = np.zeros((3,))
    z[2] = relu(np.dot(x,w[6:8]) + w[8])
    z[1] = relu(np.dot(x,w[3:5]) + w[5])
    z[0] = sigmoid(np.dot(z[1:3], w[0:2]) + w[2])
    print("\tand", z[2])
    print("\tnot x and not y", z[1])
    print("\tor", z[0])
error = error/len(Y)
print("error", "{:.2%}".format(error))

[0, 0] 1 == 1.0
	and 0.0
	not x and not y 2.0
	or 0.9996646498695336
[0, 1] 0 == 0.00034
	and 0.0
	not x and not y 0.0
	or 0.0003353501304664781
[1, 0] 0 == 0.00034
	and 0.0
	not x and not y 0.0
	or 0.0003353501304664781
[1, 1] 1 == 1.0
	and 2.0
	not x and not y 0.0
	or 0.9996646498695336
error 0.03%


### 2. (3.0)

Assuming the following loss function:

In [5]:
def loss(w, x, y):
    return (y - predict(w, x))**2/2.

Write a function that uses backpropagation to calculate:
    
$$\frac{\partial E(w, x, y)}{\partial w_0}$$

Where $E$ is the loss function defined before. Explicitely write the expressions that you derive:

$$\delta_0 = $$

$$\frac{\partial E(w, x, y)}{\partial w_0} = $$

$$\delta_0 = \frac{\delta E_l}{\delta a_0} = (z_0 - y) \sigma'(z0) $$

$$\frac{\partial E(w, x, y)}{\partial w_0} = \delta_0 z_1$$

In [23]:
def dE_dw0(w, x, y):
    # your code here
    z0 = predict(w,x)
    
    l3_error = z0 - y
    l3_delta = l3_error * sigmoid(z0) * (1-sigmoid(z0))
    
    z = np.zeros((3,))
    z[2] = relu(np.dot(x,w[6:8]) + w[8])
    z[1] = relu(np.dot(x,w[3:5]) + w[5])
    a0 = z[1]
    z[0] = sigmoid(np.dot(z[1:3], w[0:2]) + w[2])
    
    e = l3_delta * a0
    
    return e


# Numerical estimation
def num_dE_dw0(w, x, y, epsilon):
    delta = np.zeros(len(w))
    delta[0] = epsilon
    de = (loss(w + delta, x, y) - loss(w - delta, x, y)) / (2 * epsilon)
    return de

for i in range(3):
    tw = np.random.randn(9)
    tx = np.random.randn(2)
    ty = np.random.randn(1)
    print(tw, tx, ty, predict(tw, tx))
    epsilon = 0.0001
    dE = dE_dw0(tw, tx,ty)
    print(dE, num_dE_dw0(tw, tx, ty, epsilon))
    diff = np.linalg.norm(dE - num_dE_dw0(tw, tx, ty, epsilon))
    print(diff)
    if diff > epsilon:
        print("Fail")

[ 0.34147939 -0.07596938  1.92849467  1.38126845 -0.7580072   1.63895875
  1.21757044 -1.37304186  1.39475178] [-0.6306162  -0.27112827] [-1.06945823] 0.8988935482934953
[0.39393141] [0.17413764]
0.2197937756215101
Fail
[-1.83547043 -1.14791197 -0.3223096  -0.56288227  0.39647265  0.25825578
  2.17860247 -3.37189348 -0.6771984 ] [1.38653727 0.56156523] [-0.2219796] 0.30177839241137655
[0.] [0.]
0.0
[ 1.1987156   1.51255919  0.81041975 -0.94819471  0.26066137 -0.08213504
 -1.37186184 -1.07579719 -1.26908714] [ 1.15524825 -1.65500813] [0.06972757] 0.6921989439230762
[0.] [0.]
0.0


## Grader
Run the following code to grade your exam

In [15]:
def approx_equal(val1, val2):
    return abs(val1-val2) <= 0.00001

def test_dict(test, answer):
    if sorted(test.keys()) != sorted(answer.keys()): return False
    for k,v in test.items():
        if not approx_equal(v,answer[k]):
            return False
    return True

def test1():
    epsilon = 0.001
    X = [[0, 0],
     [0, 1],
     [1, 0],
     [1, 1]]
    Y = [1, 0, 0 ,1]
    for i, x in enumerate(X):
        if np.abs(predict(W1, x) - Y[i]) > epsilon: 
            return False
    return True

def num_dE_dw0(w, x, y, epsilon):
    delta = np.zeros(len(w))
    delta[0] = epsilon
    de = (loss(w + delta, x, y) - loss(w - delta, x, y)) / (2 * epsilon)
    return de

def test2():
    num_tests = 100
    epsilon = 0.0001
    for i in range(num_tests):
        tw = np.random.randn(9)
        tx = np.random.randn(2)
        ty = np.random.randn(1)
        if np.linalg.norm(dE_dw0(tw, tx,ty) - num_dE_dw0(tw, tx, ty, epsilon)) > epsilon:
            return False
    return True


def evaluate():
    score = 0 
    for test, sc in [(test1, 20), (test2, 30)]:
        if test():
            score += sc
    return score

print("Score: ", evaluate(), "/ 50")

Score:  20 / 50
